In [201]:
import pandas as pd
import numpy as np
import json

### IMPORTANT: PLEASE READ BEFORE PROCEEDING
## Data Needed

To use this notebook, make sure you have these 3 datasets in CSV format:

1. **`df`**: This is the raw annotations file exported from Label Studio.
  
2. **`video_metadata`**: A CSV version of the Google Sheet called `inner_id_to_video_metadata` containing metadata for each video.

3. **`full_raw_video_transcripts`**: A CSV version of `full_raw_video_transcripts` containing entire transcripts for each video_id

## Output

This notebook combines both datasets into a single dataframe called `merged_df`. It will then export this combined data as a CSV file named **`final_dataset.csv`** for easy access and further use.

Please modify the paths and file names below:


In [202]:
df = pd.read_csv('../data_sources/final_annotations_complete.csv')

In [203]:
video_metadata = pd.read_csv('../data_sources/inner_id_to_video_metadata.csv')

In [204]:
full_raw_video_transcripts = pd.read_csv('../data_sources/full_raw_video_transcripts.csv')

In [205]:
output_path = '../generated_dataset/complete_dataset.csv'

#### These are the field cateogorizations:
##### All fields
- `action`
- `ticker_name`
- `price`
- `quantity`
- `action_date`
- `action_source`
- `conviction_score`
---
##### Compulsory fields
- `action`
- `ticker_name`
- `action_source`
- `conviction_score`
---
##### Fields to Drop
- `agreement`
- `created_at`
- `lead_time`
- `updated_at`
- `video_url`
---
##### Supplementary Fields
- `video_title`
- `is_rec_present`
---
##### Helpful Fields
- `annotation_id`
- `annotator`
- `id`
---

## Setup and preliminary cleaning

Input dataframe in this step - `df` \
Output dataframe in this step - `df`

### Import raw annotations

In [206]:
# Already imported at the start of the notebook
df.head()

,action,action_date,action_source,agreement,annotation_id,annotator,conviction_score,created_at,id,is_rec_present,lead_time,price,quantity,ticker_name,updated_at,video_title,video_url
0,"[{""end"":158.5389470152761,""start"":109.87117723...",NaN,"[{""end"":158.5389470152761,""start"":109.87117723...",100.0,42911404,yashbhardwaj920@gmail.com,"[{""end"":158.5389470152761,""start"":109.87117723...",2024-08-17T13:52:31.686867Z,125899618,Yes,66477.470,"[{""end"":158.5389470152761,""start"":109.87117723...",NaN,"[""VERI"",""ZYXI"",""MITK"",""KRMD"",""IRMD""]",2025-01-09T17:18:33.790942Z,5 Stocks to Buy Now to Double Your Money,upload/85041/795f67ae-0CJU8R4oNFk.mp4
1,"[{""end"":280.7808211233843,""start"":166.87916727...",NaN,"[{""end"":280.7808211233843,""start"":166.87916727...",100.0,42911490,yashbhardwaj920@gmail.com,"[{""end"":280.7808211233843,""start"":166.87916727...",2024-08-17T13:58:31.269154Z,125899619,Yes,551.946,"[{""end"":280.7808211233843,""start"":166.87916727...",NaN,WYNN,2024-09-19T17:04:06.958519Z,i am selling it,upload/85041/df25da60-0Fg0YsbOzJA.mp4
2,"[{""end"":184.22808452896174,""start"":85.27745319...",NaN,"[{""end"":184.22808452896174,""start"":85.27745319...",100.0,43699384,yashbhardwaj920@gmail.com,"[{""end"":184.22808452896174,""start"":85.27745319...",2024-09-01T08:33:03.245297Z,125899620,Yes,7061.344,"[{""end"":184.22808452896174,""start"":85.27745319...",NaN,"[""GOOGL"",""CMCSA"",""META""]",2024-10-19T07:52:02.378562Z,The Best Internet Stocks for 2023 You Can Buy Now,upload/85041/00f7c6a2-0OJIHD_o59M.mp4
3,"[{""end"":642.9538446765027,""start"":39.577461060...",NaN,"[{""end"":642.9538446765027,""start"":39.577461060...",100.0,43699636,yashbhardwaj920@gmail.com,"[{""end"":642.9538446765027,""start"":39.577461060...",2024-09-01T08:58:50.522833Z,125899621,Yes,247.127,"[{""end"":642.9538446765027,""start"":39.577461060...",NaN,V,2024-09-19T17:45:57.870132Z,I Just Bought The PERFECT Dividend Stock (At A...,upload/85041/a276a34e-1Gm4A7EFYI4.mp4
4,"[{""end"":320.40466802185796,""start"":118.7247700...",NaN,"[{""end"":320.40466802185796,""start"":118.7247700...",100.0,43699888,yashbhardwaj920@gmail.com,"[{""end"":320.40466802185796,""start"":118.7247700...",2024-09-01T09:29:05.244686Z,125899622,Yes,2593.720,"[{""end"":320.40466802185796,""start"":118.7247700...",NaN,"[""SOFI"",""SPY"",""LI""]",2024-09-19T18:00:04.261058Z,🔵WARNING TO EVERYONE!!!🔵 I JUST SOLD IT ALL!!!,upload/85041/503bda30-1Lx7z_x4Rc0.mp4


### Remove specific videos

In [207]:
original_unique_id_count = df['id'].nunique()

df = df[df['id'] != 125899813] # This is the video_title with "No video" because the video was not rendering on label studio and was therefore not annotated
df = df[df['id'] != 125899678] # No transcript (see details below)
df = df[df['id'] != 125899873] # No transcript (see details below)

df = df[df['id'] != 125899907] # This is the video_title with "ERROR" because the audio was corrupted on label studio



# When this notebook was previously run, it was found that in the final dataset, video ids BQHjMXOEndc and WfqxucAaC0I had missing transcripts 
# This corresponded to the label studio id of 125899678 and 125899873 respectively 
# Since the original df exported from label studio has only the label studio id and not video id, we will remove that from df
# Note: BQHjMXOEndc was in df_transcripts but the transcript column was blank 
# Note: WfqxucAaC0I was not in df_transcripts at all

final_count = df['id'].nunique()

print("Original count of unique ids from label studio: ", original_unique_id_count)
print("Final count of unique ids after removing these videos: ", final_count)

Original count of unique ids from label studio:  292
Final count of unique ids after removing these videos:  288


### Remove row in transcripts dataframe where transcript is not there

In [208]:
print("Before removal: ", len(full_raw_video_transcripts))
full_raw_video_transcripts = full_raw_video_transcripts.dropna(subset=['transcript'])
print("After removal: ", len(full_raw_video_transcripts))

Before removal:  290
After removal:  289


## Ensuring validity of all imported data sources before proceeding

In [209]:
unique_id_count = df['id'].nunique()
print("Number of unique 'id' in label studio annotations dataframe:", unique_id_count)

unique_original_inner_id = video_metadata['original_inner_id'].nunique()
print("Number of unique original_inner_id in video_metadata dataframe:", unique_original_inner_id)

unique_transcript_video_id_count = full_raw_video_transcripts['transcript_video_id'].nunique()
print("Number of unique 'transcript_video_id' in full transcripts dataframe:", unique_transcript_video_id_count)

unique_video_ids = video_metadata['video_id'].nunique()
print("Number of unique video IDs in video_metadata dataframe:", unique_video_ids)

# Get unique video IDs from both dataframes
transcript_video_ids = set(full_raw_video_transcripts['transcript_video_id'].unique())
metadata_video_ids = set(video_metadata['video_id'].unique())

# Find video IDs that are in one set but not the other
only_in_transcripts = transcript_video_ids - metadata_video_ids
only_in_metadata = metadata_video_ids - transcript_video_ids

# Print results
print("Video IDs in transcripts but not in metadata:", only_in_transcripts)
print("Video IDs in metadata but not in transcripts:", only_in_metadata)

Number of unique 'id' in label studio annotations dataframe: 288
Number of unique original_inner_id in video_metadata dataframe: 291
Number of unique 'transcript_video_id' in full transcripts dataframe: 289
Number of unique video IDs in video_metadata dataframe: 291
Video IDs in transcripts but not in metadata: set()
Video IDs in metadata but not in transcripts: {'BQHjMXOEndc', 'WfqxucAaC0I'}


Since we are doing left joins with the main dataframe as the source "df" of label studio, we will retain the number of ids in label studio annotations dataframe

### Enclose single ticker names in a list

#### Before

In [210]:
df['ticker_name'].head()

0    ["VERI","ZYXI","MITK","KRMD","IRMD"]
1                                    WYNN
2                ["GOOGL","CMCSA","META"]
3                                       V
4                     ["SOFI","SPY","LI"]
Name: ticker_name, dtype: object

#### Apply function

In [211]:
def standardize_ticker_names(df, column_name):
    """
    Encloses single stock tickers in a list and leaves other entries unchanged.

    Parameters:
        df (pd.DataFrame): The dataframe containing the ticker names.
        column_name (str): The name of the column with ticker names.

    Returns:
        pd.DataFrame: The dataframe with the standardized ticker names.
    """
    def standardize_ticker(value):
        if pd.isna(value):  # Leave NaN values unchanged
            return value
        if isinstance(value, str) and not value.startswith("["):
            return f'["{value}"]'  # Enclose single tickers in a list-like format
        return value  # Leave lists unchanged
    
    df[column_name] = df[column_name].apply(standardize_ticker)
    return df

# Usage example:
# Assuming `data` is your DataFrame and 'ticker_name' is the column to standardize
df = standardize_ticker_names(df, 'ticker_name')



In [212]:
df['ticker_name'].head()

0    ["VERI","ZYXI","MITK","KRMD","IRMD"]
1                                ["WYNN"]
2                ["GOOGL","CMCSA","META"]
3                                   ["V"]
4                     ["SOFI","SPY","LI"]
Name: ticker_name, dtype: object

---

## Begin Validation

In [213]:
# Define the fields for validation
all_fields = ['action', 'ticker_name', 'price', 'quantity', 'action_date', 'action_source', 'conviction_score']
compulsory_fields = ['action', 'ticker_name', 'action_source', 'conviction_score']

In [214]:
# List to store failed validation records
failed_validations = []

#### Validaton checks:
1) Check if video_title is present for every video 
2) Check if all_fields are empty when is_rec_present is "No"
3) Check if compulsory fields are not empty when is_rec_present is "Yes"
4) Check if is_rec_present is not-empty for every cell in the column

In [215]:
# Function to check if video_title is present, and print 'id' if not
def check_video_title_presence(df):
    missing_video_title = df[df['video_title'].isnull()]['id']
    if not missing_video_title.empty:
        print(f"Missing video_title for IDs: {missing_video_title.tolist()}")
        for video_id in missing_video_title:
            failed_validations.append((video_id, 'check_video_title_presence'))

# Function to check if all_fields are empty when is_rec_present is "No", and print 'id' if not
def check_all_fields_empty_when_no(df):
    condition = df['is_rec_present'] == 'No'
    invalid_rows = df.loc[condition, all_fields].notnull().any(axis=1)
    if invalid_rows.any():
        print(f"All fields not empty when 'is_rec_present' is No for IDs: {df.loc[condition & invalid_rows, 'id'].tolist()}")
        for video_id in df.loc[condition & invalid_rows, 'id']:
            failed_validations.append((video_id, 'check_all_fields_empty_when_no'))


# Function to check if compulsory_fields are not empty when is_rec_present is "Yes", and print 'id' if not
def check_compulsory_fields_not_empty_when_yes(df):
    condition = df['is_rec_present'] == 'Yes'
    invalid_rows = df.loc[condition, compulsory_fields].isnull().any(axis=1)
    if invalid_rows.any():
        print(f"Compulsory fields empty when 'is_rec_present' is Yes for IDs: {df.loc[condition & invalid_rows, 'id'].tolist()}")
        for video_id in df.loc[condition & invalid_rows, 'id']:
            failed_validations.append((video_id, 'check_compulsory_fields_not_empty_when_yes'))

# Function to check if 'is_rec_present' column values are not empty
def check_is_rec_present_not_empty(df):
    missing_is_rec_present = df[df['is_rec_present'].isnull()]['id']
    if not missing_is_rec_present.empty:
        print(f"'is_rec_present' is empty for IDs: {missing_is_rec_present.tolist()}")
        for video_id in missing_is_rec_present:
            failed_validations.append((video_id, 'check_is_rec_present_not_empty'))

#### Perform validation

In [216]:
# Perform the checks and print 'id' for failures
def validate_data(df):
    check_video_title_presence(df)
    check_all_fields_empty_when_no(df)
    check_compulsory_fields_not_empty_when_yes(df)
    check_is_rec_present_not_empty(df)  # Added this new check

    # Save to CSV only if there are failed validations
    if failed_validations:
        failed_df = pd.DataFrame(failed_validations, columns=['videoId', 'failed_validation'])
        #failed_df.to_csv("../dataset/failed_validations.csv", index=False)
        #print("Failed validations saved to '../dataset/failed_validations.csv'.")
    else:
        print("No validation failures found.")

validate_data(df)

No validation failures found.


### Advanced Validation Check 1: Same length of compulsory 
Since the way the raw output is formatted, we check that if there are 5 segments, for example, all dimensions action, ticker name, action_source, and conviction score are annotated for all segments

In [217]:
# Define compulsory fields
compulsory_fields = ['action', 'ticker_name', 'action_source', 'conviction_score']

filtered_df = df.copy()


# Filter rows where is_rec_present == "Yes"
filtered_df = filtered_df[filtered_df['is_rec_present'] == 'Yes']

# Check lengths of compulsory fields for the filtered DataFrame
for index, row in filtered_df.iterrows():
    lengths = []
    for field in compulsory_fields:
        try:
            # Check if the field is already a list
            if isinstance(row[field], list):
                parsed_list = row[field]
            else:
                # Parse the JSON string if it's not already a list
                parsed_list = json.loads(row[field])
            lengths.append(len(parsed_list))
        except (json.JSONDecodeError, KeyError, TypeError):
            print(f"Error parsing field {field} in row ID {row['id']}")
            continue

    # Check if lengths differ
    if len(set(lengths)) > 1:
        print(f"Row ID {row['id']} by {row['annotator']} has differing lengths: {lengths}")


### Advanced Validation Check 2: Overlapping regions

Check that no selected regions overlap based on timestamps with one another

In [218]:
# Create a copy of df
df_adv_val_2 = df.copy()

# Deserialize the 'action' column if it contains JSON strings
df_adv_val_2['action'] = df_adv_val_2['action'].apply(lambda x: json.loads(x) if isinstance(x, str) else x)

# Function to check overlapping regions
def check_overlapping_regions(regions):
    sorted_regions = sorted(regions, key=lambda r: r['start'])
    for i in range(len(sorted_regions) - 1):
        if sorted_regions[i]['end'] > sorted_regions[i + 1]['start']:
            return True
    return False

# Check for overlapping regions in rows where is_rec_present is "Yes"
for index, row in df_adv_val_2[df_adv_val_2['is_rec_present'] == 'Yes'].iterrows():
    regions = row['action']
    if check_overlapping_regions(regions):
        print(f"Row with id {row['id']} by {row['annotator']} has overlapping regions.")

### Copy this for later use:

In [219]:
original_raw_df = df.copy()

In [220]:
original_raw_df.shape

(288, 17)

---

## Data Cleaning

Input dataframe in this step - `df` \
Output dataframe in this step - `cleaned_df`

In [221]:
# Define columns to drop
columns_to_drop = ['agreement', 'created_at', 'lead_time', 'updated_at', 'video_url']

# Drop the specified columns
df.drop(columns=columns_to_drop, inplace=True)

In [222]:
# Columns that need JSON parsing
json_columns = ['action', 'action_source', 'conviction_score', 'action_date', 'price', 'quantity']

# Parse JSON columns
for column in json_columns:
    # Use a custom function to handle JSON parsing for each cell
    df[column] = df[column].apply(lambda x: json.loads(x) if pd.notna(x) else None)

### Final step for ticker name processing: parse ticker names 

#### Before

In [223]:
df['ticker_name'].head()

0    ["VERI","ZYXI","MITK","KRMD","IRMD"]
1                                ["WYNN"]
2                ["GOOGL","CMCSA","META"]
3                                   ["V"]
4                     ["SOFI","SPY","LI"]
Name: ticker_name, dtype: object

#### Apply function

In [224]:
def parse_ticker_name(x):
    if isinstance(x, str):
        # Try to parse as JSON if it's a list-like string (e.g., '["AAPL", "TSLA"]')
        try:
            return json.loads(x)
        except json.JSONDecodeError:
            # If it's just a single string like "AAPL", wrap it in a list
            print('handling exception')
            return [x]
    return x  # Return as-is if already a list

# Apply the parsing function to the ticker_name column
df['ticker_name'] = df['ticker_name'].apply(parse_ticker_name)

#### After

In [225]:
df['ticker_name'].head()

0    [VERI, ZYXI, MITK, KRMD, IRMD]
1                            [WYNN]
2              [GOOGL, CMCSA, META]
3                               [V]
4                   [SOFI, SPY, LI]
Name: ticker_name, dtype: object

In [226]:
# Sample output from Label Studio of a JSON column
df["action"][1]

[{'end': 280.7808211233843,
  'start': 166.87916727144537,
  'labels': ['Sell'],
  'channel': 0}]

### Cleaned data for is_rec_present == "Yes"

In [227]:
def clean_data(raw_data):
    # Initialize list to store each extracted recommendation
    recommendations = []

    # Filter rows where 'is_rec_present' is 'Yes'
    filtered_data = raw_data[raw_data['is_rec_present'] == 'Yes']

    # Iterate through each row in the filtered dataset
    for _, row in filtered_data.iterrows():
        # Directly use action, action_source, and conviction_score fields as lists
        actions = row['action']
        action_sources = row['action_source']
        conviction_scores = row['conviction_score']

        # Optional fields: action_date, price, quantity
        action_dates = row['action_date'] if row['action_date'] is not None else []
        prices = row['price'] if row['price'] is not None else []
        quantities = row['quantity'] if row['quantity'] is not None else []

        # Ticker names, assume chronological order
        ticker_names = row['ticker_name'] if row['ticker_name'] is not None else []

        # Iterate over each action and match based on timestamps
        for i, action in enumerate(actions):
            # Extract start and end timestamps
            start_time = action['start']
            end_time = action['end']

            # Get the label from the action field
            action_label = action.get('labels', [None])[0]

            # Find the matching action_source based on exact start and end timestamps
            matching_action_source = next((src for src in action_sources if src['start'] == start_time and src['end'] == end_time), None)
            action_source_label = matching_action_source.get('labels', [None])[0] if matching_action_source else None

            # Find the matching conviction_score based on exact start and end timestamps
            matching_conviction_score = next((score for score in conviction_scores if score['start'] == start_time and score['end'] == end_time), None)
            conviction_score_label = matching_conviction_score.get('labels', [None])[0] if matching_conviction_score else None

            # Get the ticker name in chronological order
            ticker_name = ticker_names[i] if i < len(ticker_names) else None

            # Find the optional action_date based on exact timestamps
            matching_action_date = next((date for date in action_dates if date['start'] == start_time and date['end'] == end_time), None) if action_dates else None
            action_date_value = matching_action_date.get('datetime') if matching_action_date else None

            # Find the optional price and quantity based on exact timestamps and extract from 'number' key
            matching_price = next((price for price in prices if price['start'] == start_time and price['end'] == end_time), None) if prices else None
            price_value = matching_price.get('number') if matching_price else None

            matching_quantity = next((quantity for quantity in quantities if quantity['start'] == start_time and quantity['end'] == end_time), None) if quantities else None
            quantity_value = matching_quantity.get('number') if matching_quantity else None

            # Append the extracted data as a single recommendation row
            recommendations.append({
                'id': row['id'],
                'derived_inner_id': row['id'] - 125899617,  # derived inner id calculation (label studio)
                'start': start_time,
                'end': end_time,
                'action': action_label,
                'action_source': action_source_label,
                'conviction_score': conviction_score_label,
                'ticker_name': ticker_name,
                'action_date': action_date_value,
                'price': price_value,
                'quantity': quantity_value,
                'video_title': row['video_title'],
                'annotation_id': row['annotation_id'],  # annotation_id at the end
                'annotator': row['annotator'],           # annotator at the end
                'is_rec_present': row['is_rec_present']  # is_rec_present at the end
            })
    recommendations_df = pd.DataFrame(recommendations)
    return recommendations_df

In [228]:
# Create the cleaned DataFrame and save in csv
df_cleaned_yes_recommendations = clean_data(df)
df_cleaned_yes_recommendations.head()

,id,derived_inner_id,start,end,action,action_source,conviction_score,ticker_name,action_date,price,quantity,video_title,annotation_id,annotator,is_rec_present
0,125899618,1,109.871177,158.538947,Buy,Title,2,VERI,None,7.75,NaN,5 Stocks to Buy Now to Double Your Money,42911404,yashbhardwaj920@gmail.com,Yes
1,125899618,1,197.798669,267.774387,Buy,Selected region,2,ZYXI,None,15.99,NaN,5 Stocks to Buy Now to Double Your Money,42911404,yashbhardwaj920@gmail.com,Yes
2,125899618,1,465.293375,514.698535,Buy,Title,2,MITK,None,12.51,NaN,5 Stocks to Buy Now to Double Your Money,42911404,yashbhardwaj920@gmail.com,Yes
3,125899618,1,515.435925,559.602978,Buy,Selected region,3,KRMD,None,7.65,NaN,5 Stocks to Buy Now to Double Your Money,42911404,yashbhardwaj920@gmail.com,Yes
4,125899618,1,560.518396,614.246246,Buy,Selected region,2,IRMD,None,22.11,NaN,5 Stocks to Buy Now to Double Your Money,42911404,yashbhardwaj920@gmail.com,Yes


### Cleaned data for is_rec_present == "No"

In [229]:
df_raw_no_recommendations = df[df['is_rec_present'] == 'No']

# Check for duplicate IDs
duplicate_ids = df_raw_no_recommendations[df_raw_no_recommendations.duplicated(subset='id', keep=False)]

if not duplicate_ids.empty:
    print("Duplicate IDs found:")
    print(duplicate_ids)
else:
    print("No duplicate IDs found.")

No duplicate IDs found.


In [230]:
def reformat_dataframe(df):
    recommendations = []
    for _, row in df.iterrows():
        recommendations.append({
            'id': row['id'],
            'derived_inner_id': int(row['id']) - 125899617,  # derived inner id calculation
            'start': None,  # Replace with actual logic if available
            'end': None,    # Replace with actual logic if available
            'action': row['action'],
            'action_source': row['action_source'],
            'conviction_score': row['conviction_score'],
            'ticker_name': row['ticker_name'],
            'action_date': row['action_date'],
            'price': row['price'],
            'quantity': row['quantity'],
            'video_title': row['video_title'],
            'annotation_id': row['annotation_id'],  # annotation_id at the end
            'annotator': row['annotator'],           # annotator at the end
            'is_rec_present': row['is_rec_present']  # is_rec_present at the end
        })
    df_processed = pd.DataFrame(recommendations)
    return df_processed

# Apply the function
df_cleaned_no_recommendations = reformat_dataframe(df_raw_no_recommendations)
df_cleaned_no_recommendations.head()


,id,derived_inner_id,start,end,action,action_source,conviction_score,ticker_name,action_date,price,quantity,video_title,annotation_id,annotator,is_rec_present
0,125899627,10,None,None,None,None,None,NaN,None,None,None,Buy Dividend Stocks & Don't Stop,43691656,1anandmk@gmail.com,No
1,125899628,11,None,None,None,None,None,NaN,None,None,None,I Don't Buy INFLATION Projections!,43691686,1anandmk@gmail.com,No
2,125899634,17,None,None,None,None,None,NaN,None,None,None,To Buy Or Not To Buy Stocks With Crash And Rec...,43773520,1anandmk@gmail.com,No
3,125899647,30,None,None,None,None,None,NaN,None,None,None,"FACEBOOK STOCK, FAANG - NEWS - WHAT IS GOING O...",44424172,1anandmk@gmail.com,No
4,125899648,31,None,None,None,None,None,NaN,None,None,None,Panic Buying The Stock Market Is Going On Righ...,44424319,1anandmk@gmail.com,No


### Making `cleaned_df` by combining df_cleaned_yes_recommendations and df_cleaned_no_recommendations

In [231]:
# Check if columns are identical
if list(df_cleaned_yes_recommendations.columns) == list(df_cleaned_no_recommendations.columns):
    # Combine the dataframes if columns match
    cleaned_df = pd.concat([df_cleaned_yes_recommendations, df_cleaned_no_recommendations], ignore_index=True)
    print("DataFrames combined successfully.")
else:
    print("Column mismatch detected. Check the column names and order before combining.")


DataFrames combined successfully.


/var/folders/qz/8l46blhj3lsg6h3hgljjlmsr0000gn/T/ipykernel_3906/4146533504.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  cleaned_df = pd.concat([df_cleaned_yes_recommendations, df_cleaned_no_recommendations], ignore_index=True)


In [232]:
cleaned_df.head()

,id,derived_inner_id,start,end,action,action_source,conviction_score,ticker_name,action_date,price,quantity,video_title,annotation_id,annotator,is_rec_present
0,125899618,1,109.871177,158.538947,Buy,Title,2,VERI,None,7.75,NaN,5 Stocks to Buy Now to Double Your Money,42911404,yashbhardwaj920@gmail.com,Yes
1,125899618,1,197.798669,267.774387,Buy,Selected region,2,ZYXI,None,15.99,NaN,5 Stocks to Buy Now to Double Your Money,42911404,yashbhardwaj920@gmail.com,Yes
2,125899618,1,465.293375,514.698535,Buy,Title,2,MITK,None,12.51,NaN,5 Stocks to Buy Now to Double Your Money,42911404,yashbhardwaj920@gmail.com,Yes
3,125899618,1,515.435925,559.602978,Buy,Selected region,3,KRMD,None,7.65,NaN,5 Stocks to Buy Now to Double Your Money,42911404,yashbhardwaj920@gmail.com,Yes
4,125899618,1,560.518396,614.246246,Buy,Selected region,2,IRMD,None,22.11,NaN,5 Stocks to Buy Now to Double Your Money,42911404,yashbhardwaj920@gmail.com,Yes


## Data Merging - Human Annotations and Channel Metadata

Input dataframe in this step - `cleaned_df` \
Metadata Input dataframe in this step - `inner_id_to_video_metadata` \
Output dataframe in this step - `merged_df`

To consolidate our data sources, we'll merge two primary datasets:

1.  **`cleaned_df`**: This dataset contains cleaned human annotations from Label Studio, with each row representing a singular stock recommendation.
2.  **`inner_id_to_video_metadata`**: This dataset maps `original_inner_id` to multiple columns which are video metadata columns, providing metadata details for each video associated with the recommendations.

### Objective

Our aim is to have each row in the merged dataframe represent a single stock recommendation, complete with all relevant video metadata. By joining `cleaned_df` with `inner_id_to_video_metadata` based on `original_inner_id`, each recommendation in `cleaned_df` will be enriched with corresponding video metadata, creating a comprehensive view for analysis.


#### Import video metadata dataset

In [233]:
# Already imported at the start of the notebook
video_metadata.head()

,video_id,original_inner_id,original_video_title,publishedAt,channelId,channelTitle,videoDescription,tags,defaultAudioLanguage,duration,...,viewCount,likeCount,favoriteCount,commentCount,comments,channelDescription,channelViewCount,channelSubscriberCount,videoCount,channelCategory
0,0CJU8R4oNFk,1,5 Stocks to Buy Now to Double Your Money,2020-10-07 14:45:00+00:00,UCbKdotYtcY9SxoU8CYAXdvg,"Let's Talk Money! with Joseph Hogue, CFA",I never say this but these stocks will double ...,"['how to double your money fast', 'stocks to b...",en,628,...,72157,2661.0,0,168.0,"["".I don’t who, but someone actually needs to ...",Welcome to your chance to create the financial...,43592318,641000,1168,Category 1
1,0Fg0YsbOzJA,2,i am selling it,2018-01-19 21:37:06+00:00,UCnMn36GT_H0X-w5_ckLtlgQ,Financial Education,Link to join StockHub free investing discord s...,['financial education'],en,564,...,14765,340.0,0,128.0,"['Hey Bud, did you sell off all your Wynn Reso...",My name is Jeremy Lefebvre and I created the F...,114243548,732000,2573,Category 1
2,0OJIHD_o59M,3,The Best Internet Stocks for 2023 You Can Buy Now,2022-10-19 17:45:00+00:00,UCbKdotYtcY9SxoU8CYAXdvg,"Let's Talk Money! with Joseph Hogue, CFA",Special Announcement! I’m putting my favorite ...,"['bow tie nation', 'joseph hogue', 'bow tie in...",en,659,...,15122,743.0,0,58.0,"[""😯Don't forget to follow the Bow Tie Index (B...",Welcome to your chance to create the financial...,43592318,641000,1168,Category 1
3,1Gm4A7EFYI4,4,I Just Bought The PERFECT Dividend Stock (At A...,2024-01-10 17:45:00+00:00,UC4SK8IQ_u14VjUE8KtugSmw,Ryne Williams,📊 GET MY DIVIDEND PORTFOLIO SPREADSHEET (FREE)...,"['dividend stocks', 'dividend income', 'divide...",en-US,659,...,14705,615.0,0,231.0,['What was the most recent stock you added to ...,This channel is all about reaching financial f...,7923235,58000,700,Category 1
4,1Lx7z_x4Rc0,5,🔵WARNING TO EVERYONE!!!🔵 I JUST SOLD IT ALL!!!,2024-01-12 22:21:54+00:00,UCoMzWLaPjDJBbipihD694pQ,Stock Moe,"🚀 *TODAY! Moe's Stock Course Code ""CASH"" $70 O...","['best growth stocks', 'best small cap stocks ...",en-US,697,...,16065,890.0,0,77.0,"[""Come on over today and take your next step i...",The Stock Moe YouTube channel tries to bring t...,85876827,625000,2710,Category 1


In [234]:
print("Numbers of rows in cleaned_df where each row is a recommendation (counting yes and no): ", len(cleaned_df))
print("Number of rows in video metadata df where each row is a video: ", len(video_metadata))

Numbers of rows in cleaned_df where each row is a recommendation (counting yes and no):  760
Number of rows in video metadata df where each row is a video:  291


In [235]:
# Merge the dataframes on the inner IDs, keeping only the rows from cleaned_df
merged_df = cleaned_df.merge(video_metadata, how='left', left_on='derived_inner_id', right_on='original_inner_id')
print("When I merge cleaned df and video metadata df, I have,", len(merged_df),"rows")

When I merge cleaned df and video metadata df, I have, 760 rows


In [236]:
merged_df.head()

,id,derived_inner_id,start,end,action,action_source,conviction_score,ticker_name,action_date,price,...,viewCount,likeCount,favoriteCount,commentCount,comments,channelDescription,channelViewCount,channelSubscriberCount,videoCount,channelCategory
0,125899618,1,109.871177,158.538947,Buy,Title,2,VERI,None,7.75,...,72157,2661.0,0,168.0,"["".I don’t who, but someone actually needs to ...",Welcome to your chance to create the financial...,43592318,641000,1168,Category 1
1,125899618,1,197.798669,267.774387,Buy,Selected region,2,ZYXI,None,15.99,...,72157,2661.0,0,168.0,"["".I don’t who, but someone actually needs to ...",Welcome to your chance to create the financial...,43592318,641000,1168,Category 1
2,125899618,1,465.293375,514.698535,Buy,Title,2,MITK,None,12.51,...,72157,2661.0,0,168.0,"["".I don’t who, but someone actually needs to ...",Welcome to your chance to create the financial...,43592318,641000,1168,Category 1
3,125899618,1,515.435925,559.602978,Buy,Selected region,3,KRMD,None,7.65,...,72157,2661.0,0,168.0,"["".I don’t who, but someone actually needs to ...",Welcome to your chance to create the financial...,43592318,641000,1168,Category 1
4,125899618,1,560.518396,614.246246,Buy,Selected region,2,IRMD,None,22.11,...,72157,2661.0,0,168.0,"["".I don’t who, but someone actually needs to ...",Welcome to your chance to create the financial...,43592318,641000,1168,Category 1


In [237]:
full_raw_video_transcripts.head()

,transcript_video_id,transcript
0,0CJU8R4oNFk,"Hey Bowtie Nation, Joseph Hogue here with the..."
1,0Fg0YsbOzJA,So we have decided to start selling out of Wy...
2,0OJIHD_o59M,"Hey Bowtie Nation, Joseph Hogue here and a ve..."
3,1Gm4A7EFYI4,So I've had my eye on a few different stocks ...
4,1Lx7z_x4Rc0,Family we absolutely dominated it today. If y...


In [238]:
# Merge the dataframes
merged_df_with_transcripts = merged_df.merge(
    full_raw_video_transcripts, 
    how="left", 
    left_on="video_id", 
    right_on="transcript_video_id"
)

# Identify missing video_ids
missing_video_ids = merged_df_with_transcripts[merged_df_with_transcripts["transcript"].isnull()]["video_id"]

# Print missing video IDs
if not missing_video_ids.empty:
    print("The following video_ids are missing transcripts:", missing_video_ids.tolist())

# Drop the redundant merge column
merged_df_with_transcripts = merged_df_with_transcripts.drop(columns=["transcript_video_id"])

#### Change column ordering for final dataframe if needed

In [239]:
# Get the list of columns in the DataFrame
columns = merged_df_with_transcripts.columns.tolist()

# Remove 'video_id' from its current position
columns.remove('video_id')

# Insert 'video_id' as the third column
columns.insert(2, 'video_id')

# Reorder the DataFrame with the new column order
merged_df_with_transcripts = merged_df_with_transcripts[columns]


In [240]:
merged_df_with_transcripts.head()

,id,derived_inner_id,video_id,start,end,action,action_source,conviction_score,ticker_name,action_date,...,likeCount,favoriteCount,commentCount,comments,channelDescription,channelViewCount,channelSubscriberCount,videoCount,channelCategory,transcript
0,125899618,1,0CJU8R4oNFk,109.871177,158.538947,Buy,Title,2,VERI,None,...,2661.0,0,168.0,"["".I don’t who, but someone actually needs to ...",Welcome to your chance to create the financial...,43592318,641000,1168,Category 1,"Hey Bowtie Nation, Joseph Hogue here with the..."
1,125899618,1,0CJU8R4oNFk,197.798669,267.774387,Buy,Selected region,2,ZYXI,None,...,2661.0,0,168.0,"["".I don’t who, but someone actually needs to ...",Welcome to your chance to create the financial...,43592318,641000,1168,Category 1,"Hey Bowtie Nation, Joseph Hogue here with the..."
2,125899618,1,0CJU8R4oNFk,465.293375,514.698535,Buy,Title,2,MITK,None,...,2661.0,0,168.0,"["".I don’t who, but someone actually needs to ...",Welcome to your chance to create the financial...,43592318,641000,1168,Category 1,"Hey Bowtie Nation, Joseph Hogue here with the..."
3,125899618,1,0CJU8R4oNFk,515.435925,559.602978,Buy,Selected region,3,KRMD,None,...,2661.0,0,168.0,"["".I don’t who, but someone actually needs to ...",Welcome to your chance to create the financial...,43592318,641000,1168,Category 1,"Hey Bowtie Nation, Joseph Hogue here with the..."
4,125899618,1,0CJU8R4oNFk,560.518396,614.246246,Buy,Selected region,2,IRMD,None,...,2661.0,0,168.0,"["".I don’t who, but someone actually needs to ...",Welcome to your chance to create the financial...,43592318,641000,1168,Category 1,"Hey Bowtie Nation, Joseph Hogue here with the..."


## Final Checks

### Final Check 1: For each recommendation, we have `video_title` and `is_rec_present`

In [241]:
constant_columns = ['video_title', 'is_rec_present']

# Filter rows where any of the constant_columns have null values
null_rows = merged_df_with_transcripts[merged_df_with_transcripts[constant_columns].isnull().any(axis=1)]

# Check if there are any null rows and print the corresponding 'id' and 'annotator'
if not null_rows.empty:
    print("Rows with null values in specified columns:")
    print(null_rows[['id', 'annotator']])
else:
    print("All values in the specified columns are non-null.")

All values in the specified columns are non-null.


### Final Check 2: For each `is_rec_present = No`, `all_annotated_columns_for_yes` is Null

In [242]:
all_annotated_columns_for_yes = ['start', 'end', 'action', 'action_source', 'conviction_score', 'ticker_name', 'action_date', 'price', 'quantity']


# Filter rows where is_rec_present is 'No'
invalid_rows = merged_df_with_transcripts[
    (merged_df_with_transcripts['is_rec_present'] == 'No') & 
    (merged_df_with_transcripts[all_annotated_columns_for_yes].notnull().any(axis=1))
]

# Check and print invalid rows
if not invalid_rows.empty:
    for index, row in invalid_rows.iterrows():
        print(f"Row ID: {row['id']}, Annotator: {row['annotator']}")
else:
    print("All rows with 'is_rec_present' = 'No' have null values for the specified columns.")

All rows with 'is_rec_present' = 'No' have null values for the specified columns.


### Final Check 3: For each `is_prec_present = Yes`, check that all `compulsory_fields` non-null values
This ensures for each recommendation, we have all the compulsory fields

In [243]:
compulsory_fields = ['action', 'ticker_name', 'action_source', 'conviction_score']

# Filter rows where 'is_rec_present' is 'Yes'
filtered_df = merged_df_with_transcripts[merged_df_with_transcripts['is_rec_present'] == 'Yes']

# Check for nulls in compulsory fields
invalid_rows = filtered_df[filtered_df[compulsory_fields].isnull().any(axis=1)]

# Print 'id' and 'annotator' for rows with nulls in compulsory fields
if not invalid_rows.empty:
    print(invalid_rows[['id', 'annotator']])
else:
    print("All rows with 'is_rec_present' as 'Yes' have non-null compulsory fields.")


All rows with 'is_rec_present' as 'Yes' have non-null compulsory fields.


## check final videos and final recommendations count

## For Raw

Explanation: There are 4 compulsory fields: action, ticker name, action source, and convicton score. \
3 of these are stored as JSONs and 1 is stored as a normal list. \
The length of these represent the number of recommendations \
Ideally, the length of all of these should be the same for each row. \
But, to be safe, we will take the max length of all 4, and store it as the number of recommendations for that id (I am using id and not video_id because video_id is not present in the raw label studio exported dataframe in the raw dataframe)

In [244]:
# Define compulsory fields
compulsory_fields = ['action', 'ticker_name', 'action_source', 'conviction_score']

# Create a new list to store the id and raw_rec_count for each row
results = []

# Iterate through each row in the original DataFrame
for index, row in original_raw_df.iterrows():
    lengths = []
    for field in compulsory_fields:
        try:
            # Check if the field is already a list
            if isinstance(row[field], list):
                parsed_list = row[field]
            else:
                # Parse the JSON string if it's not already a list
                parsed_list = json.loads(row[field])
            # Append the length of the list to lengths
            lengths.append(len(parsed_list))
        except (json.JSONDecodeError, TypeError, KeyError):
            # Handle cases where parsing fails or the field is missing
            lengths.append(0)
    
    # Calculate the maximum length for the current row
    max_length = max(lengths) if lengths else 0
    
    # Append the result as a dictionary
    results.append({'id': row['id'], 'raw_rec_count': max_length})

# Create a new DataFrame from the results
df_raw_count = pd.DataFrame(results)

# Display the resulting DataFrame
df_raw_count.head()

,id,raw_rec_count
0,125899618,5
1,125899619,1
2,125899620,3
3,125899621,1
4,125899622,3


In [245]:
unique_id_count = df_raw_count['id'].nunique()
print("Number of unique ids in raw dataframe: ", unique_id_count)

total_raw_rec_count = df_raw_count['raw_rec_count'].sum()
print("Number of recommendations in raw dataframe:", total_raw_rec_count)

Number of unique ids in raw dataframe:  288
Number of recommendations in raw dataframe: 687


## For Cleaned

In [246]:
# Print the number of unique "id"
print("Number of unique 'id' in final cleaned dataframe:", merged_df_with_transcripts["id"].nunique())

# Print the number of unique "video_id"
print("Number of unique 'video_id' in final cleaned dataframe:", merged_df_with_transcripts["video_id"].nunique())


Number of unique 'id' in final cleaned dataframe: 288
Number of unique 'video_id' in final cleaned dataframe: 288


The code below counts a row as a recommendaton if is_rec_present == "Yes".

In [247]:
# Initialize the count
count = 0

# Iterate through each row in the dataframe
for _, row in merged_df_with_transcripts.iterrows():
    if row['is_rec_present'] == 'Yes':
        count += 1

# Print the final count
print("Number of Yes recommendations in cleaned final dataframe:", count)

Number of Yes recommendations in cleaned final dataframe: 687


The code groups merged_df_with_transcripts by 'id', checks if any row in each group has 'is_rec_present' as 'No', and assigns 0 if found. Otherwise, it counts the number of records in the group. The final result is stored in a new DataFrame (df_clean_count) that contains 'id' and its corresponding count of valid records.

In [248]:
# Group by 'id' and check the conditions
def compute_clean_rec_count(group):
    if 'No' in group['is_rec_present'].values:
        return 0
    else:
        return len(group)

# Create the new dataframe
df_clean_count = (
    merged_df_with_transcripts
    .groupby('id')
    .apply(compute_clean_rec_count)
    .reset_index(name='clean_rec_count')
)

/var/folders/qz/8l46blhj3lsg6h3hgljjlmsr0000gn/T/ipykernel_3906/759916078.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(compute_clean_rec_count)


In [249]:
df_clean_count

,id,clean_rec_count
0,125899618,5
1,125899619,1
2,125899620,3
3,125899621,1
4,125899622,3
...,...,...
283,125899904,0
284,125899905,6
285,125899906,1
286,125899908,0


In [250]:
unique_id_count = df_clean_count['id'].nunique()
print(unique_id_count)

total_raw_rec_count = df_clean_count['clean_rec_count'].sum()
print(total_raw_rec_count)

288
687


In [251]:
df_both = pd.merge(df_raw_count, df_clean_count, on='id', how='outer')
df_both

,id,raw_rec_count,clean_rec_count
0,125899618,5,5
1,125899619,1,1
2,125899620,3,3
3,125899621,1,1
4,125899622,3,3
...,...,...,...
283,125899904,0,0
284,125899905,6,6
285,125899906,1,1
286,125899908,0,0


In [252]:
df_different = df_both[df_both['clean_rec_count'] != df_both['raw_rec_count']]
df_different

,id,raw_rec_count,clean_rec_count


---

### Final modifications to dataframe before exporting

### Creating `youtube_video_url`

In [253]:
# Construct the YouTube video URL
merged_df_with_transcripts['youtube_video_url'] = 'https://www.youtube.com/watch?v=' + merged_df_with_transcripts['video_id']

---

### Export Final Dataframe

#### This is the dataframe completed BUT WITHOUT SEGMENTWISE TRANSCRIPTS

In [254]:
print(merged_df_with_transcripts.shape)
# I am not exporting this right now
# Just keeping it
merged_df_with_transcripts

(760, 38)


,id,derived_inner_id,video_id,start,end,action,action_source,conviction_score,ticker_name,action_date,...,favoriteCount,commentCount,comments,channelDescription,channelViewCount,channelSubscriberCount,videoCount,channelCategory,transcript,youtube_video_url
0,125899618,1,0CJU8R4oNFk,109.871177,158.538947,Buy,Title,2,VERI,None,...,0,168.0,"["".I don’t who, but someone actually needs to ...",Welcome to your chance to create the financial...,43592318,641000,1168,Category 1,"Hey Bowtie Nation, Joseph Hogue here with the...",https://www.youtube.com/watch?v=0CJU8R4oNFk
1,125899618,1,0CJU8R4oNFk,197.798669,267.774387,Buy,Selected region,2,ZYXI,None,...,0,168.0,"["".I don’t who, but someone actually needs to ...",Welcome to your chance to create the financial...,43592318,641000,1168,Category 1,"Hey Bowtie Nation, Joseph Hogue here with the...",https://www.youtube.com/watch?v=0CJU8R4oNFk
2,125899618,1,0CJU8R4oNFk,465.293375,514.698535,Buy,Title,2,MITK,None,...,0,168.0,"["".I don’t who, but someone actually needs to ...",Welcome to your chance to create the financial...,43592318,641000,1168,Category 1,"Hey Bowtie Nation, Joseph Hogue here with the...",https://www.youtube.com/watch?v=0CJU8R4oNFk
3,125899618,1,0CJU8R4oNFk,515.435925,559.602978,Buy,Selected region,3,KRMD,None,...,0,168.0,"["".I don’t who, but someone actually needs to ...",Welcome to your chance to create the financial...,43592318,641000,1168,Category 1,"Hey Bowtie Nation, Joseph Hogue here with the...",https://www.youtube.com/watch?v=0CJU8R4oNFk
4,125899618,1,0CJU8R4oNFk,560.518396,614.246246,Buy,Selected region,2,IRMD,None,...,0,168.0,"["".I don’t who, but someone actually needs to ...",Welcome to your chance to create the financial...,43592318,641000,1168,Category 1,"Hey Bowtie Nation, Joseph Hogue here with the...",https://www.youtube.com/watch?v=0CJU8R4oNFk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,125899895,278,Y61LJ2Pwsgw,NaN,NaN,None,None,None,NaN,None,...,0,53.0,"[""I'm on Penn and Gan"", ""This appears like a w...",My goal is to help YOU be a better investor & ...,4800938,71400,515,Category 1,"Hey guys, Eric here, Mr. Fired Up Wealth. I w...",https://www.youtube.com/watch?v=Y61LJ2Pwsgw
756,125899896,279,Y8iGpTUtrvI,NaN,NaN,None,None,None,NaN,None,...,0,47.0,"['Which are your favorite ETFs to buy, growth ...",Welcome to your chance to create the financial...,43592318,641000,1168,Category 1,"Hey Bowtie Nation, Joseph Hogue here and grow...",https://www.youtube.com/watch?v=Y8iGpTUtrvI
757,125899904,287,zxji3hEMdOM,NaN,NaN,None,None,None,NaN,None,...,0,44.0,['This could have been great but you missed a ...,Welcome to your chance to create the financial...,43592318,641000,1168,Category 1,The average cost to rent over a lifetime is w...,https://www.youtube.com/watch?v=zxji3hEMdOM
758,125899908,291,_KdRpU1CMzY,NaN,NaN,None,None,None,NaN,None,...,0,71.0,"[""Shib is running! Let's go...hit that subscri...",The Stock Moe YouTube channel tries to bring t...,85876827,625000,2710,Category 1,Crypto is exploding up. We are seeing a few t...,https://www.youtube.com/watch?v=_KdRpU1CMzY


### Step 1:
#### We modify the final dataframe by dropping all rows except: `video_id`, `youtube_video_url`, `start`, `end`, `is_rec_present`
##### Note: we are doing this for segmentwise whisper

In [255]:
# Copy the DataFrame
df_recommendation_timestamps = merged_df_with_transcripts.copy()

# Keep only the required columns
df_recommendation_timestamps = df_recommendation_timestamps[[
    "video_id", "youtube_video_url", "start", "end", "is_rec_present"
]]

In [256]:
df_recommendation_timestamps

,video_id,youtube_video_url,start,end,is_rec_present
0,0CJU8R4oNFk,https://www.youtube.com/watch?v=0CJU8R4oNFk,109.871177,158.538947,Yes
1,0CJU8R4oNFk,https://www.youtube.com/watch?v=0CJU8R4oNFk,197.798669,267.774387,Yes
2,0CJU8R4oNFk,https://www.youtube.com/watch?v=0CJU8R4oNFk,465.293375,514.698535,Yes
3,0CJU8R4oNFk,https://www.youtube.com/watch?v=0CJU8R4oNFk,515.435925,559.602978,Yes
4,0CJU8R4oNFk,https://www.youtube.com/watch?v=0CJU8R4oNFk,560.518396,614.246246,Yes
...,...,...,...,...,...
755,Y61LJ2Pwsgw,https://www.youtube.com/watch?v=Y61LJ2Pwsgw,NaN,NaN,No
756,Y8iGpTUtrvI,https://www.youtube.com/watch?v=Y8iGpTUtrvI,NaN,NaN,No
757,zxji3hEMdOM,https://www.youtube.com/watch?v=zxji3hEMdOM,NaN,NaN,No
758,_KdRpU1CMzY,https://www.youtube.com/watch?v=_KdRpU1CMzY,NaN,NaN,No


### Step 2:
#### We save this dataframe as `recommendation_timestamps_only.csv` 

In [257]:
# Save to CSV
# We are not running this - THIS WAS A ONETIME RUN 
# THE FILE YOU SEE ON RECORD RIGHT NOW IS THE EXACT FILE USED FOR SEGMENTWISE WHISPER INPUT
#df_recommendation_timestamps.to_csv("../intermediate_data/recommendation_timestamps_only.csv", index=False)

#print("CSV file saved successfully as df_recommendation_timestamps.csv")

### Step 3:
#### This CSV file is used as input into the segmentwise whisper code 

In [258]:
##### SEGMENTWISE WHISPER CODE IN OTHER FILE XYZ.PY 

### Step 4:
#### The CSV file is modified and one extra column called `segment_transcript` is added to it. Lets import this file.

In [259]:
df_recommendation_timestamps_with_s_transcripts = pd.read_csv("../intermediate_data/recommendation_timestamps_with_s_transcripts.csv")

In [260]:
df_recommendation_timestamps_with_s_transcripts

,video_id,youtube_video_url,start,end,is_rec_present,segment_transcript
0,0CJU8R4oNFk,https://www.youtube.com/watch?v=0CJU8R4oNFk,109.871177,158.538947,Yes,these in a minute. First up is $225 million d...
1,0CJU8R4oNFk,https://www.youtube.com/watch?v=0CJU8R4oNFk,197.798669,267.774387,Yes,"up and make your first deposit. Zynex, ticker..."
2,0CJU8R4oNFk,https://www.youtube.com/watch?v=0CJU8R4oNFk,465.293375,514.698535,Yes,"Our next stock, $500 million dollar Midic Sys..."
3,0CJU8R4oNFk,https://www.youtube.com/watch?v=0CJU8R4oNFk,515.435925,559.602978,Yes,next few years. Our next pick here is a healt...
4,0CJU8R4oNFk,https://www.youtube.com/watch?v=0CJU8R4oNFk,560.518396,614.246246,Yes,"Our next stock to buy here, Iradomid, ticker ..."
...,...,...,...,...,...,...
756,Y8iGpTUtrvI,https://www.youtube.com/watch?v=Y8iGpTUtrvI,NaN,NaN,No,NaN
757,zxji3hEMdOM,https://www.youtube.com/watch?v=zxji3hEMdOM,NaN,NaN,No,NaN
758,_fdC-a7isQ4,https://www.youtube.com/watch?v=_fdC-a7isQ4,NaN,NaN,No,NaN
759,_KdRpU1CMzY,https://www.youtube.com/watch?v=_KdRpU1CMzY,NaN,NaN,No,NaN


### Step 5:
#### Take our main dataframe `merged_df_with_transcripts` and add the column `segment_transcript` from the imported dataframe above

Note: we are merging the column based on `video_id`, `start`, `end`, `is_rec_present`

In [261]:
# Fill NaN values with a default value before rounding (e.g., -1, or another appropriate value)
merged_df_with_transcripts['start_rounded'] = merged_df_with_transcripts['start'].fillna(-1).round().astype(int)
merged_df_with_transcripts['end_rounded'] = merged_df_with_transcripts['end'].fillna(-1).round().astype(int)

df_recommendation_timestamps_with_s_transcripts['start_rounded'] = df_recommendation_timestamps_with_s_transcripts['start'].fillna(-1).round().astype(int)
df_recommendation_timestamps_with_s_transcripts['end_rounded'] = df_recommendation_timestamps_with_s_transcripts['end'].fillna(-1).round().astype(int)

# Perform the merge using the rounded columns
complete_dataset = merged_df_with_transcripts.merge(
    df_recommendation_timestamps_with_s_transcripts[['video_id', 'start_rounded', 'end_rounded', 'is_rec_present', 'segment_transcript']], 
    on=['video_id', 'start_rounded', 'end_rounded', 'is_rec_present'], 
    how='left'
)

# Drop the temporary columns after merging
complete_dataset.drop(columns=['start_rounded', 'end_rounded'], inplace=True)

In [262]:
complete_dataset

,id,derived_inner_id,video_id,start,end,action,action_source,conviction_score,ticker_name,action_date,...,commentCount,comments,channelDescription,channelViewCount,channelSubscriberCount,videoCount,channelCategory,transcript,youtube_video_url,segment_transcript
0,125899618,1,0CJU8R4oNFk,109.871177,158.538947,Buy,Title,2,VERI,None,...,168.0,"["".I don’t who, but someone actually needs to ...",Welcome to your chance to create the financial...,43592318,641000,1168,Category 1,"Hey Bowtie Nation, Joseph Hogue here with the...",https://www.youtube.com/watch?v=0CJU8R4oNFk,these in a minute. First up is $225 million d...
1,125899618,1,0CJU8R4oNFk,197.798669,267.774387,Buy,Selected region,2,ZYXI,None,...,168.0,"["".I don’t who, but someone actually needs to ...",Welcome to your chance to create the financial...,43592318,641000,1168,Category 1,"Hey Bowtie Nation, Joseph Hogue here with the...",https://www.youtube.com/watch?v=0CJU8R4oNFk,"up and make your first deposit. Zynex, ticker..."
2,125899618,1,0CJU8R4oNFk,465.293375,514.698535,Buy,Title,2,MITK,None,...,168.0,"["".I don’t who, but someone actually needs to ...",Welcome to your chance to create the financial...,43592318,641000,1168,Category 1,"Hey Bowtie Nation, Joseph Hogue here with the...",https://www.youtube.com/watch?v=0CJU8R4oNFk,"Our next stock, $500 million dollar Midic Sys..."
3,125899618,1,0CJU8R4oNFk,515.435925,559.602978,Buy,Selected region,3,KRMD,None,...,168.0,"["".I don’t who, but someone actually needs to ...",Welcome to your chance to create the financial...,43592318,641000,1168,Category 1,"Hey Bowtie Nation, Joseph Hogue here with the...",https://www.youtube.com/watch?v=0CJU8R4oNFk,next few years. Our next pick here is a healt...
4,125899618,1,0CJU8R4oNFk,560.518396,614.246246,Buy,Selected region,2,IRMD,None,...,168.0,"["".I don’t who, but someone actually needs to ...",Welcome to your chance to create the financial...,43592318,641000,1168,Category 1,"Hey Bowtie Nation, Joseph Hogue here with the...",https://www.youtube.com/watch?v=0CJU8R4oNFk,"Our next stock to buy here, Iradomid, ticker ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,125899895,278,Y61LJ2Pwsgw,NaN,NaN,None,None,None,NaN,None,...,53.0,"[""I'm on Penn and Gan"", ""This appears like a w...",My goal is to help YOU be a better investor & ...,4800938,71400,515,Category 1,"Hey guys, Eric here, Mr. Fired Up Wealth. I w...",https://www.youtube.com/watch?v=Y61LJ2Pwsgw,NaN
756,125899896,279,Y8iGpTUtrvI,NaN,NaN,None,None,None,NaN,None,...,47.0,"['Which are your favorite ETFs to buy, growth ...",Welcome to your chance to create the financial...,43592318,641000,1168,Category 1,"Hey Bowtie Nation, Joseph Hogue here and grow...",https://www.youtube.com/watch?v=Y8iGpTUtrvI,NaN
757,125899904,287,zxji3hEMdOM,NaN,NaN,None,None,None,NaN,None,...,44.0,['This could have been great but you missed a ...,Welcome to your chance to create the financial...,43592318,641000,1168,Category 1,The average cost to rent over a lifetime is w...,https://www.youtube.com/watch?v=zxji3hEMdOM,NaN
758,125899908,291,_KdRpU1CMzY,NaN,NaN,None,None,None,NaN,None,...,71.0,"[""Shib is running! Let's go...hit that subscri...",The Stock Moe YouTube channel tries to bring t...,85876827,625000,2710,Category 1,Crypto is exploding up. We are seeing a few t...,https://www.youtube.com/watch?v=_KdRpU1CMzY,NaN


In [263]:
print(complete_dataset.shape)
complete_dataset

(760, 39)


,id,derived_inner_id,video_id,start,end,action,action_source,conviction_score,ticker_name,action_date,...,commentCount,comments,channelDescription,channelViewCount,channelSubscriberCount,videoCount,channelCategory,transcript,youtube_video_url,segment_transcript
0,125899618,1,0CJU8R4oNFk,109.871177,158.538947,Buy,Title,2,VERI,None,...,168.0,"["".I don’t who, but someone actually needs to ...",Welcome to your chance to create the financial...,43592318,641000,1168,Category 1,"Hey Bowtie Nation, Joseph Hogue here with the...",https://www.youtube.com/watch?v=0CJU8R4oNFk,these in a minute. First up is $225 million d...
1,125899618,1,0CJU8R4oNFk,197.798669,267.774387,Buy,Selected region,2,ZYXI,None,...,168.0,"["".I don’t who, but someone actually needs to ...",Welcome to your chance to create the financial...,43592318,641000,1168,Category 1,"Hey Bowtie Nation, Joseph Hogue here with the...",https://www.youtube.com/watch?v=0CJU8R4oNFk,"up and make your first deposit. Zynex, ticker..."
2,125899618,1,0CJU8R4oNFk,465.293375,514.698535,Buy,Title,2,MITK,None,...,168.0,"["".I don’t who, but someone actually needs to ...",Welcome to your chance to create the financial...,43592318,641000,1168,Category 1,"Hey Bowtie Nation, Joseph Hogue here with the...",https://www.youtube.com/watch?v=0CJU8R4oNFk,"Our next stock, $500 million dollar Midic Sys..."
3,125899618,1,0CJU8R4oNFk,515.435925,559.602978,Buy,Selected region,3,KRMD,None,...,168.0,"["".I don’t who, but someone actually needs to ...",Welcome to your chance to create the financial...,43592318,641000,1168,Category 1,"Hey Bowtie Nation, Joseph Hogue here with the...",https://www.youtube.com/watch?v=0CJU8R4oNFk,next few years. Our next pick here is a healt...
4,125899618,1,0CJU8R4oNFk,560.518396,614.246246,Buy,Selected region,2,IRMD,None,...,168.0,"["".I don’t who, but someone actually needs to ...",Welcome to your chance to create the financial...,43592318,641000,1168,Category 1,"Hey Bowtie Nation, Joseph Hogue here with the...",https://www.youtube.com/watch?v=0CJU8R4oNFk,"Our next stock to buy here, Iradomid, ticker ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,125899895,278,Y61LJ2Pwsgw,NaN,NaN,None,None,None,NaN,None,...,53.0,"[""I'm on Penn and Gan"", ""This appears like a w...",My goal is to help YOU be a better investor & ...,4800938,71400,515,Category 1,"Hey guys, Eric here, Mr. Fired Up Wealth. I w...",https://www.youtube.com/watch?v=Y61LJ2Pwsgw,NaN
756,125899896,279,Y8iGpTUtrvI,NaN,NaN,None,None,None,NaN,None,...,47.0,"['Which are your favorite ETFs to buy, growth ...",Welcome to your chance to create the financial...,43592318,641000,1168,Category 1,"Hey Bowtie Nation, Joseph Hogue here and grow...",https://www.youtube.com/watch?v=Y8iGpTUtrvI,NaN
757,125899904,287,zxji3hEMdOM,NaN,NaN,None,None,None,NaN,None,...,44.0,['This could have been great but you missed a ...,Welcome to your chance to create the financial...,43592318,641000,1168,Category 1,The average cost to rent over a lifetime is w...,https://www.youtube.com/watch?v=zxji3hEMdOM,NaN
758,125899908,291,_KdRpU1CMzY,NaN,NaN,None,None,None,NaN,None,...,71.0,"[""Shib is running! Let's go...hit that subscri...",The Stock Moe YouTube channel tries to bring t...,85876827,625000,2710,Category 1,Crypto is exploding up. We are seeing a few t...,https://www.youtube.com/watch?v=_KdRpU1CMzY,NaN


---

In [264]:
complete_dataset.drop('annotator', axis=1, inplace=True)

In [265]:
# Handle Label Studio Glitch - on label studiom annotators were not able to make "price" or "quantity" blank, by deleting the numerical value.
# Instead, it would only go to 0. This is confusing for the final dataset, hence we will replace 0 with NaN for all price and quantity columns.

complete_dataset['price']    = complete_dataset['price'].replace(0, np.nan)
complete_dataset['quantity'] = complete_dataset['quantity'].replace(0, np.nan)

In [266]:
complete_dataset.to_csv(output_path, index = False)

---